In [1]:
# Utils
import pandas as pd
import numpy as np
from copy import deepcopy
from sklearn.preprocessing import MinMaxScaler
import warnings
from pathlib import Path

import keras
import tensorflow as tf
from tensorflow import keras

##
warnings.filterwarnings("ignore")

In [2]:
# Configs
sample_data_path = Path('../data/01_samples')
data_path =  Path('../data/02_outputs')
model_path = Path('../models/rnn_v1')

In [3]:
# Função para padronizar os nomes entre as bases de partidas e ranking ECO
country_dict = {'North Ireland':'Northern Ireland',
                'Qatar':'United Arab Emirates',
                'Bosnia/Herzeg':'Bosnia and Herzegovina',
                'Czechia':'Czech Republic',
                'Ireland':'Republic of Ireland',
                'Trinidad/Tob': 'Trinidad and Tobago',
                'Serbia/Mont':'Serbia and Montenegro'}

In [4]:
# Funções
def create_teams_df(data):
        
    data_home = data[['year','home_team_name','home_n_participation','year_n']]
    data_away = data[['year','away_team_name','away_n_participation','year_n']]
    data_home.rename(columns={'home_team_name':'team', 'home_n_participation':'n_participation'}, inplace=True)
    data_away.rename(columns={'away_team_name':'team', 'away_n_participation':'n_participation'}, inplace=True)
    
    data_final = pd.concat([data_home, data_away], ignore_index=True)
    data_final.drop_duplicates(inplace=True)
    
    return data_final

## Tratamento de dados

# Função para padronizar os nomes entre as bases de partidas e ranking ECO
country_dict = {'North Ireland':'Northern Ireland',
                'Qatar':'United Arab Emirates',
                'Bosnia/Herzeg':'Bosnia and Herzegovina',
                'Czechia':'Czech Republic',
                'Ireland':'Republic of Ireland',
                'Trinidad/Tob': 'Trinidad and Tobago',
                'Serbia/Mont':'Serbia and Montenegro'}

def fix_country_name(data):
    
    if data['team'] in country_dict.keys():
        return country_dict[data['team']]
        
    else:
        return data['team']
    
def fix_missing_score(data, feature):
    
    base_col = '{f}_team_name'
    
    if data[base_col.format(f=feature)] == 'West Germany' and data['year'] == 1990:
        return 2082
    
    elif data[base_col.format(f=feature)] == 'Serbia and Montenegro' and data['year'] == 2006:
        return 1690
    
    else:
        return data['{f}_score'.format(f=feature)]

def fix_missing_rank(data, feature):
    
    base_col = '{f}_team_name'
    
    if data[base_col.format(f=feature)] == 'West Germany' and data['year'] == 1990:
        return 1
    
    elif data[base_col.format(f=feature)] == 'Serbia and Montenegro' and data['year'] == 2006:
        return 40
    
    else:
        return data['{f}_rank'.format(f=feature)]
    

In [5]:
# Carregando os dados e Preparando os dados
df = pd.read_csv(data_path / '01_matches_eco_score.csv')

df.drop(['match_date', 'best_rank_winner', 'team_score_diff',
         'draw', 'home_winner','draw', 'away_winner'],
         axis=1, inplace=True)

df['year_n'] = df['year']
df.dropna(inplace=True)
df.head()

,home_team_name,away_team_name,home_team_score,away_team_score,year,home_rank,home_score,away_rank,away_score,home_n_participation,away_n_participation,rank_diff,score_diff,year_n
0,France,Mexico,4,1,1930,35.0,1566.0,41.0,1497.0,0,0,-6.0,69.0,1930
1,United States,Belgium,3,0,1930,14.0,1762.0,31.0,1593.0,0,0,-17.0,169.0,1930
2,Yugoslavia,Brazil,2,1,1930,25.0,1658.0,9.0,1890.0,0,0,16.0,-232.0,1930
3,Romania,Peru,3,1,1930,37.0,1544.0,42.0,1496.0,0,0,-5.0,48.0,1930
4,Argentina,France,1,0,1930,1.0,2062.0,35.0,1566.0,0,0,-34.0,496.0,1930


In [6]:
df = df[['year','home_team_name','away_team_name','home_n_participation','away_n_participation','year_n']]

In [7]:
df = create_teams_df(df)

In [ ]:
grupo_a = ['Netherlands', 'Senegal', 'Ecuador','United Arab Emirates']
grupo_e = ['England','United States','Iran','Wales']
grupo_b = ['Argentina','Poland','Mexico','Saudi Arabia']
grupo_f = ['Australia','Tunisia','Bosnia and Herzegovina','Iran']
grupo_c = ['Colombia','Greece','Ivory Coast','Japan']
grupo_g = ['Germany','United States','Portugal','Ghana']
grupo_d = ['Costa Rica','Uruguay','Italy','England']
grupo_h = ['Belgium','Algeria','Russia','South Korea']

In [23]:
df[df['team'] == 'Tunisia']

,year,team,n_participation,year_n
271,1978,Tunisia,0,1978
609,2002,Tunisia,2,2002
658,2006,Tunisia,3,2006
848,2018,Tunisia,4,2018
1427,1998,Tunisia,1,1998


In [ ]:
countries = ['Netherlands', 'Senegal','Ecuador','United Arab Emirates','England','United States','Iran',
             'Wales', 'Argentina', 'Poland','Mexico','Saudi Arabia', 'France', 'Australia','Tunisia',
             '']